# Image Processing Pipeline for the Accepted Manuscript:   
Quirin D. Strotzer, Rehab N. Khalid, Sara De Giorgi, Aman B. Patel, Michael H. Lev, Rajiv Gupta, Robert W. Regenhardt
Predictive Value of CT Perfusion in Spatially and Volumetrically Identifying Ischemic Penumbra Against Final Infarct Size in Anterior Circulation Stroke With and Without Successful Reperfusion
American Journal of Neuroradiology Jun 2025, ajnr.A8876; DOI: 10.3174/ajnr.A8876

## Prerequisites

Filenames:

Baseline Imaging:   
{CASE ID}_NCCT.nii.gz (baseline non contrast head CT)   
{CASE ID}_Perfusion_Structural.nii.gz (baseline perfusion CT structural image, generated using syngo.via)      
{CASE ID}_Perfusion_RGB.nii.gz (baseline perfusion CT RGB image with core and penumbra visualized according to chosen thresholds, generated using syngo.via)   

Follow-up Imaging:   
{CASE ID}_NCCT_FollowUP.nii.gz (follow up non contrast CT depicting infarct)   

cases.txt: Case IDs (one per line)

In [ ]:
from pathlib import Path
import os
import shutil
from tqdm import tqdm
import SimpleITK as sitk
from totalsegmentator.python_api import totalsegmentator
import torch
import numpy as np
import nibabel as nib
from nipype.interfaces.ants import Registration
from nilearn.image import resample_to_img

DATA_DIR = Path("/your/data/directory") # with subfolders for each case

cases = [line.rstrip('\n') for line in open(os.path.join(DATA_DIR, 'cases.txt'))]

## Preprocess non-contrast CTs

### Resampling to 512x512 in-plane

In [ ]:
def resample_nifti_image(input_file, output_file, new_size=(512, 512), new_spacing_z=5.0):
    image = sitk.ReadImage(input_file, sitk.sitkInt16)

    original_size = image.GetSize()
    original_spacing = image.GetSpacing()

    new_spacing_x = original_spacing[0] * (original_size[0] / new_size[0])
    new_spacing_y = original_spacing[1] * (original_size[1] / new_size[1])
    new_spacing = (new_spacing_x, new_spacing_y, new_spacing_z)

    new_size_z = int(original_size[2] * (original_spacing[2] / new_spacing_z))
    new_size_full = (new_size[0], new_size[1], new_size_z)

    resampler = sitk.ResampleImageFilter()
    resampler.SetOutputSpacing(new_spacing)
    resampler.SetSize(new_size_full)
    resampler.SetOutputDirection(image.GetDirection())
    resampler.SetOutputOrigin(image.GetOrigin())
    resampler.SetTransform(sitk.Transform())
    resampler.SetInterpolator(sitk.sitkLinear)

    resampled_image = resampler.Execute(image)

    sitk.WriteImage(resampled_image, output_file)


for c in tqdm(cases):

    try:
        resample_nifti_image(
            os.path.join(DATA_DIR, c, f"{c}_NCCT.nii.gz"),
            os.path.join(DATA_DIR, c, f"{c}_NCCT_REFORMAT.nii.gz")
        )
    except RuntimeError:
        print(c, "NCCT")

    try:    
        resample_nifti_image(
            os.path.join(DATA_DIR, c, f"{c}_NCCT_FollowUP.nii.gz"),
            os.path.join(DATA_DIR, c, f"{c}_NCCT_FollowUP_REFORMAT.nii.gz")
        )
    except RuntimeError:
        print(c, "FU NCCT")

### Skull Stripping

In [ ]:
for c in tqdm(cases):
    totalsegmentator(
        os.path.join(DATA_DIR, c, f"{c}_NCCT_REFORMAT.nii.gz"),
        os.path.join(DATA_DIR, c, f"{c}_NCCT_REFORMAT_Segmentation.nii.gz"),
        ml=True,
        #fast=True,
        roi_subset=["brain"]
    )


    totalsegmentator(
        os.path.join(DATA_DIR, c, f"{c}_NCCT_FollowUP_REFORMAT.nii.gz"),
        os.path.join(DATA_DIR, c, f"{c}_NCCT_FollowUP_REFORMAT_Segmentation.nii.gz"),
        ml=True,
        #fast=True,
        roi_subset=["brain"]
    )

### Apply Segmentation Masks

In [ ]:
def load_and_apply_mask(image_path, mask_path):
    
    image = sitk.ReadImage(image_path)
    mask = sitk.ReadImage(mask_path)
    mask = mask > 0
    filled_mask = sitk.BinaryFillhole(mask)
    filled_mask = sitk.Cast(filled_mask, image.GetPixelID())
    masked_image = image * filled_mask
    masked_image = sitk.Cast(masked_image, sitk.sitkFloat32)
    masked_image.SetOrigin((0, 0, 0))
    masked_image.SetDirection((1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0))

    return masked_image

for c in tqdm(cases):
    masked_image = load_and_apply_mask(
        os.path.join(DATA_DIR, c, f"{c}_NCCT_FollowUP_REFORMAT.nii.gz"),
        os.path.join(DATA_DIR, c, f"{c}_NCCT_FollowUP_REFORMAT_Segmentation.nii.gz"),
        )
    sitk.WriteImage(masked_image, os.path.join(DATA_DIR, c, f"{c}_NCCT_FollowUP_REFORMAT_BET.nii.gz"))

    masked_image = load_and_apply_mask(
        os.path.join(DATA_DIR, c, f"{c}_NCCT_REFORMAT.nii.gz"),
        os.path.join(DATA_DIR, c, f"{c}_NCCT_REFORMAT_Segmentation.nii.gz"),
        )
    sitk.WriteImage(masked_image, os.path.join(DATA_DIR, c, f"{c}_NCCT_REFORMAT_BET.nii.gz"))

## Preprocess Perfusion Images

### Convert Structural Perfusion image from RGB tu HU

In [ ]:
def convert_rgb_nifti_to_grayscale_hu_adjusted(input_filename, output_filename):

    img_rgb = sitk.ReadImage(input_filename)
    img_array = sitk.GetArrayFromImage(img_rgb)
    
    # Calculate the grayscale image using luminance formula
    img_gray_array = np.dot(img_array[..., :3], [0.299, 0.587, 0.114])
    img_gray_array_hu = img_gray_array - 128

    # Z-score normalization
    hu_mean = np.mean(img_gray_array_hu)
    hu_std = np.std(img_gray_array_hu)
    img_gray_array_hu = (img_gray_array_hu - hu_mean) / hu_std
    
    img_gray_hu = sitk.GetImageFromArray(img_gray_array_hu.astype(np.float32))
    img_gray_hu.SetSpacing(img_rgb.GetSpacing())
    img_gray_hu.SetOrigin((0,0,0))
    img_gray_hu.SetDirection((1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0))

    sitk.WriteImage(img_gray_hu, output_filename)

for c in tqdm(cases):

    convert_rgb_nifti_to_grayscale_hu_adjusted(
        os.path.join(DATA_DIR, c, f"{c}_Perfusion_Structural.nii.gz"),
        os.path.join(DATA_DIR, c, f"{c}_Perfusion_Structural_HU.nii.gz"),
    )

### Extract Penumbra Segmentation

#### Reformat
- Orientation of exported RGB images had to be adjusted (in our case)

In [ ]:
def update_z_spacing_and_mirror_image(input_image_path, output_image_path):
    image = sitk.ReadImage(input_image_path)
    # print(image.GetDirection())

    spacing = list(image.GetSpacing())

    spacing[2] = 5.0

    image.SetSpacing(spacing)

    image = sitk.Flip(image, flipAxes=[False, False, True])
    image.SetOrigin((0, 0, 0))
    image.SetDirection((1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0))
    # print(image.GetDirection())
    sitk.WriteImage(image, output_image_path)

for c in tqdm(cases):
    update_z_spacing_and_mirror_image(
        os.path.join(DATA_DIR, c, f"{c}_Perfusion_RGB.nii.gz"),
        os.path.join(DATA_DIR, c, f"{c}_Perfusion_RGB_reformat.nii.gz"),
)

#### Extract Segmentation
- RGB Perfusion maps had red (infarct core) and yellow (ischemic penumbra) voxels as calculated by pre-selected thresholds

In [ ]:
def create_label_mask(nifti_image_path):

    img = sitk.ReadImage(nifti_image_path)
    data = sitk.GetArrayFromImage(img)

    label_mask = np.zeros(data.shape[:-1], dtype=np.uint8)

    #print(type(data))
    #print(data.shape)

    red_channel = data[..., 0]
    green_channel = data[..., 1]
    blue_channel = data[..., 2]

    is_red = (red_channel == 255) & (green_channel == 0) & (blue_channel == 0)
    is_yellow = (red_channel == 255) & (green_channel == 255) & (blue_channel == 0)

    label_mask[is_red] = 1
    label_mask[is_yellow] = 2

    new_img = sitk.GetImageFromArray(label_mask)
    new_img.CopyInformation(img)

    return new_img

for c in tqdm(cases):
    label_mask = create_label_mask(os.path.join(DATA_DIR, c, f"{c}_Perfusion_RGB_reformat.nii.gz"))
    sitk.WriteImage(
        label_mask,
        os.path.join(DATA_DIR, c, f"{c}_Perfusion_Segmentation.nii.gz")
    )

## Co-Registration

In [ ]:
def co_register(fixed_image, moving_image, out_image):
    reg = Registration()
    reg.inputs.fixed_image = fixed_image
    reg.inputs.moving_image = moving_image
    reg.inputs.output_transform_prefix = "transform"
    reg.inputs.transforms = ["Rigid", "Affine", "SyN"]
    reg.inputs.num_threads = 10
    reg.inputs.float = True
    reg.inputs.transform_parameters = [(2.0,), (2.0,), (0.25, 3.0, 0.0)]
    reg.inputs.number_of_iterations = [[1000, 200], [1000, 200], [100, 50, 30]]
    reg.inputs.dimension = 3
    reg.inputs.write_composite_transform = False
    reg.inputs.collapse_output_transforms = True
    reg.inputs.initialize_transforms_per_stage = False
    reg.inputs.output_inverse_warped_image = False
    reg.inputs.metric = ["Mattes"] * 3
    reg.inputs.metric_weight = [1] * 3
    reg.inputs.radius_or_number_of_bins = [32] * 3
    reg.inputs.sampling_strategy = ["Random", "Random", None]
    reg.inputs.sampling_percentage = [0.05, 0.05, None]
    reg.inputs.convergence_threshold = [1.0e-8, 1.0e-8, 1.0e-9]
    reg.inputs.convergence_window_size = [20] * 3
    reg.inputs.smoothing_sigmas = [[1, 0], [1, 0], [2, 1, 0]]
    reg.inputs.sigma_units = ["vox"] * 3
    reg.inputs.shrink_factors = [[2, 1], [2, 1], [3, 2, 1]]
    reg.inputs.use_estimate_learning_rate_once = [True] * 3
    reg.inputs.use_histogram_matching = [True] * 3
    reg.inputs.output_warped_image = out_image
    reg.run()

for c in tqdm(cases):

    print(c)

    try:

        co_register(
            os.path.join(DATA_DIR, c, f"{c}_Perfusion_Structural_HU.nii.gz"),
            os.path.join(DATA_DIR, c, f"{c}_NCCT_REFORMAT_BET.nii.gz"),
            os.path.join(DATA_DIR, c, f"{c}_NCCT_REFORMAT_BET_COREG.nii.gz"),
        )

        co_register(
            os.path.join(DATA_DIR, c, f"{c}_Perfusion_Structural_HU.nii.gz"),
            os.path.join(DATA_DIR, c, f"{c}_NCCT_FollowUP_REFORMAT_BET.nii.gz"),
            os.path.join(DATA_DIR, c, f"{c}_NCCT_FollowUP_REFORMAT_BET_COREG.nii.gz"),
        )
    
    except Exception as e:
        print(e)

## Resample to Perfusion RGB Space
- Otherwise the positioning of the two segmentations might not be correct

In [ ]:
images_to_resample = [
"NCCT_FollowUP_REFORMAT_BET_COREG",
"Perfusion_Structural_HU",
"NCCT_REFORMAT_BET_COREG",
]

for c in tqdm(cases):

    for img in images_to_resample:

        fixed_path = os.path.join(DATA_DIR, c, f"{c}_Perfusion_RGB_reformat.nii.gz")
        moving_path = os.path.join(DATA_DIR, c, f"{c}_{img}.nii.gz")
        img1 = nib.load(fixed_path)
        img2 = nib.load(moving_path)

        # Copy the affine (sform) from img1 to img2
        resampled_img2 = resample_to_img(img2, img1, interpolation='nearest')

        adjusted_img2_path = os.path.join(DATA_DIR, c, f"{c}_{img}_RESAMPLE.nii.gz")
        nib.save(resampled_img2, adjusted_img2_path)

## Segment Follow up Defect and Correct Perfusion Segmentation

- This is a manual step. Use a segmentation tool of your choice to:    
(I) Segment the ischemic defect on the follow up non contrast CT.   
(II) Inspect and, if needed, remove artifacts from the perfusion segmentation.  